# Native Traefik

https://github.com/comafire/skp_n4e_traefik

Traefik (https://doc.traefik.io/traefik/)는 최근 각광 받고 있는 Reverse Proxy 입니다. 특히 Traefik 의 경우는 Let's Encrypt를 통해 자동 SSL 인증 업데이트 기능을 지원하고, 설정 변경 시 동적 리로딩을 지원하기 때문에 좀 더 편리하게 트래픽의 라우팅을 조정할 수 있습니다. 

## 설치

먼저, SKP2를 사용하기 위한 기본 설치를 아래 링크를 통해 진행합니다.

https://comafire.github.io/dataplatform/skp.html

skp_n4e_traefik 프로젝트를 다운받아 아래 순서대로 설치합니다.

```bash
> cd ~/skp
> git clone https://github.com/comafire/skp_n4e_traefik.git
> cd skp_n4e_traefik
```

이제 .envrc.template 파일을 복사해 자신의 환경에 맞게 환경 변수들을 수정해 줍니다.

```bash
# BASE (주의: 환경 변수의 기준 값으로 변수 값 내에 다른 환경 변수를 참조하면 안됨)
export SKP_USER="skp"
export SKP_USERID=$(id -u $SKP_USER)
export SKP_USER_HOME="/home/skp"
export SKP_HOME="/home/skp/skp"

# LOCALE
export LOCALE="ko_KR.UTF-8" # or "en_US.UTF-8"

# PYTHON
export PYENV_PYVER="3.8.9"
export PYENV_VENV="skp-n4e-traefik"
export PATH="$HOME/.pyenv/bin:${PATH}"
eval "$(pyenv init --path)"
eval "$(pyenv init -)"
eval "$(pyenv virtualenv-init -)"
export PYENV_VIRTUALENV_DISABLE_PROMPT=1
pyenv activate ${PYENV_VENV} 

# NODEJS - N
export N_PREFIX="${SKP_USER_HOME}/n"; [[ :$PATH: == *":$N_PREFIX/bin:"* ]] || PATH+=":$N_PREFIX/bin"  # Added by n-install (see http://git.io/n-install-repo).

# TRAEFIK
export TRAEFIK_VER="2.4.8"
export TRAEFIK_HOME="${SKP_HOME}/skp_n4e_traefik"
export TRAEFIK_NAME="skp-n4e-traefik"
export TRAEFIK_LOG="true"
export TRAEFIK_LOG_LEVEL="debug"
export TRAEFIK_ENTRYPOINTS_WEB="true"
export TRAEFIK_ENTRYPOINTS_WEB_ADDRESS=":80"
export TRAEFIK_ENTRYPOINTS_WEBSECURE="true"
export TRAEFIK_ENTRYPOINTS_WEBSECURE_ADDRESS=":443"
export TRAEFIK_CERTIFICATESRESOLVERS_LETSENCRYPT="true"
export TRAEFIK_CERTIFICATESRESOLVERS_LETSENCRYPT_ACME_EMAIL="YOUR_EMAIL"
export TRAEFIK_CERTIFICATESRESOLVERS_LETSENCRYPT_ACME_STORAGE="${TRAEFIK_HOME}/var/acme.json"
export TRAEFIK_CERTIFICATESRESOLVERS_LETSENCRYPT_ACME_HTTPCHALLENGE="true"
export TRAEFIK_CERTIFICATESRESOLVERS_LETSENCRYPT_ACME_HTTPCHALLENGE_ENTRYPOINT="web"
export TRAEFIK_PROVIDERS_FILE_DIRECTORY="${TRAEFIK_HOME}/etc"
export TRAEFIK_PROVIDERS_FILE_FILENAME="traefik_dynamic.yml"
```

traefik 에서 사용될 $TRAEFIK_HOME/etc/traefik_dynamic.yml 설정 파일을 자신의 환경에 맞게 수정해 줍니다.

```
http:
  routers:
    nginx-skp-com:
      rule: "Host(`nginx.skp.com`)"
      service: nginx-skp-com
      priority: 1000
      # # for http
      # entryPoints:
      #   - web
      # for https
      tls:
        certResolver: letsencrypt

  services:
    nginx-skp-com:
      loadBalancer:
        servers:
          - url: "http://182.168.1.22:80"
```

이제 설정된 환경 변수를 이용하여 traefik 를 설치 및 기동 할 수 있습니다.

```bash
> direnv allow
> pyenv install $PYENV_PYVER
> pyenv virtualenv $PYENV_PYVER $PYENV_VENV
> pyenv virtualenvs
> direnv allow
> pip install --upgrade pip 
> pip install setuptools invoke wheel

> ./bin/skp.sh -l
Available tasks:

  traefik.install
  traefik.restart
  traefik.start
  traefik.stop
  
> ./bin/skp.sh traefik.install
> ./bin/skp.sh traefik.start
```
